In [1]:
import sys
import os

# Workaround to make packages work in both Jupyter notebook and Python
MODULE_ROOT_NAME = "AgeEstimator"
MODULE_PATHS = [
    os.path.abspath(os.path.join('..')),
    os.path.abspath(os.path.join('../..')),
    os.path.abspath(os.path.join('../../..'))
]
MODULE_PATHS = list(
    filter(lambda x: x.endswith(MODULE_ROOT_NAME), MODULE_PATHS))
MODULE_PATH = MODULE_PATHS[0] if len(MODULE_PATHS) == 1 else ""
if MODULE_PATH not in sys.path:
    sys.path.append(MODULE_PATH)
    
from server.data.dataset import DataLoader
from server.models.cnn.model import get_model, OLD_WEIGHTS_PATH, BEST_WEIGHTS_PATH, LABEL_MAPPING, get_models, N_CLASSES, IMAGE_SIZE

Using TensorFlow backend.
/home/daniel/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pandas/core/series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [2]:
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import tensorflow as tf
import matplotlib.image as img
import numpy as np
import pandas
import multiprocessing

## Global variables

In [3]:
batch_size = 64

## Pocessing

In [7]:
def get_label_to_category_map():
    unique_labels = list(set(LABEL_MAPPING.values()))
    category_map = {class_label: inx for inx, class_label in enumerate(unique_labels)}
    category_map_r = {inx: class_label for inx, class_label in enumerate(unique_labels)}
    return category_map, category_map_r

In [8]:
def normalize_label(y):
    category_map, _ = get_label_to_category_map()
    normalize = lambda x:category_map[LABEL_MAPPING[x]]
    labels = np.vectorize(normalize)(y)
    return to_categorical(labels, N_CLASSES)

In [9]:
def get_img_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    
    valid_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    return train_datagen, valid_datagen, test_datagen

In [10]:
def to_generator(datagen, dataframe, directory, batch_size=batch_size):
    g = datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col="FilePath",
        y_col="Age",
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
#         class_mode='sparse',
        class_mode="categorical"
    )

    # Convert to tf.data to better utilize multiprocessing
    n_class = len(np.unique(np.array(dataframe["Age"])))
    tf_g = tf.data.Dataset.from_generator(lambda: g,
        output_types=(tf.float32, tf.float32),
        output_shapes=(
            tf.TensorShape([None, IMAGE_SIZE[0], IMAGE_SIZE[1], 3]), 
            tf.TensorShape([None, 55])
        )
    )

    return tf_g

## Sampling and train/valid Split

In [11]:
def get_dataframe(x, y, name, sample_size=0):
    # Stack to [[img, label], ...] matrix
    stk = np.column_stack((x, y))
    
    # Save as csv
    np.savetxt("%s.csv" % (name), stk, fmt="%s", delimiter=",", comments="", header="FilePath,Age")
    
    # `flow_from_dataframe` requires loading labels as string
    df = pandas.read_csv("./%s.csv" % (name), dtype=str)
    
    return df if sample_size == 0 else df.sample(n=sample_size)

In [12]:
def split_train_valid(df):
    train_df = df.sample(frac=0.9)
    validation_df = df.drop(train_df.index)
    return train_df, validation_df

## Training Utilities

In [20]:
def get_callbacks(log_dir):
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

    # Don't waste our time/resource on bad training
    es = EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1,
        patience=20)
    
    tb = TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,
        write_graph=True,
        write_grads=False,
        write_images=False,
        embeddings_freq=0,
        embeddings_layer_names=None,
        embeddings_metadata=None,
        embeddings_data=None,
        update_freq='epoch')
    
    # Save the best weight seen so far
    mc = ModelCheckpoint(
        BEST_WEIGHTS_PATH,
#         monitor='val_loss',
#         mode='min',
        monitor='val_categorical_accuracy',
        mode='max',
        verbose=1,
        save_weights_only=True,
        save_best_only=True)
    
    # Modify the best score for retrains
    mc.best = 0.10575
    
    # Try to get rid of local minimum
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=6,
        min_lr=0.000001)
    
    return [mc, es, tb, reduce_lr]

In [14]:
def get_log_dir():
    log_i = 0
    log_dir = "logs/run_"
    
    while os.path.exists(log_dir + str(log_i)):
        log_i += 1

    return log_dir + str(log_i)

In [15]:
def compare_results(y_true, y_predict, top_n=5):
    r"""Compare the last 10 result of top 5 prediction and its label."""
    y_hat = y_predict.argsort(axis=1)[:,-top_n:]
    print(y_hat[-10:])
    print(y_true[-10:])

## Training

### Train a lot of models

Train with a small portion of our dataset to compare the performace of the combinations of hyperparameters, so we can decide which model should be trained with a larger epochs.

In [16]:
def train_many(train_generator, valid_generator, train_len, valid_len):
    epochs = 20
    models = get_models()
    
    for m in models:
        model_name, optimizer, model = m
        print("== Training %s ==" % model_name)

        model.compile(loss="categorical_crossentropy", optimizer=optimizer, \
                      metrics=["categorical_accuracy"])

        log_dir = get_log_dir()
        callbacks = get_callbacks(log_dir + "/%s" % model_name)

        model.fit(
            x=train_generator,
            steps_per_epoch=train_len // batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=valid_generator,
            validation_steps=valid_len // batch_size,
            callbacks=callbacks,
            workers=max(2, multiprocessing.cpu_count() - 2),
            use_multiprocessing=True
        )

        model.save_weights("%s_weight.hdf5" % model_name)

    return model

### Train the finalized model

In [21]:
def train(x, y):
    epochs = 1000
    
    optimizer = Nadam(lr=0.006, beta_1=0.9, beta_2=0.999)

    model = get_model()
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, \
        metrics=["categorical_accuracy"])
    
    if os.path.exists(BEST_WEIGHTS_PATH):
        model.load_weights(BEST_WEIGHTS_PATH)
        print("best weight [%s] loaded." % BEST_WEIGHTS_PATH)
    elif os.path.exists(OLD_WEIGHTS_PATH):
        model.load_weights(OLD_WEIGHTS_PATH)
        print("old weight [%s] loaded." % OLD_WEIGHTS_PATH)
    else:
        print("fresh start.")
            
    log_dir = get_log_dir()
    callbacks = get_callbacks(log_dir)

    train_len = round(len(x) * 0.9)
    
    model.fit(
        x=x,
        y=y,
        epochs=epochs,
        verbose=1,
        validation_split=0.1,
        shuffle=True,
        callbacks=callbacks,
        workers=max(2, multiprocessing.cpu_count() - 2),
        use_multiprocessing=True
    )
    
    model.save_weights(OLD_WEIGHTS_PATH)

    return model

### Entry point

In [22]:
def main(sample_size=0, is_final_model=True):
    dl = DataLoader()
    use_bottleneck_features = True
    x_train, y_train = dl.load_train(use_bottleneck_features)
    x_test, y_test = dl.load_test(use_bottleneck_features)
    
    # Discretizate the continuous age into ordinal labels and map it with one-hot encoding
    y_train = normalize_label(y_train)
    y_test = normalize_label(y_test)
    
#     # The size is too large, so build a csv file for (image_filename/label) mapping
#     train_df = get_dataframe(x_train, y_train, "train", sample_size=sample_size)
#     train_df, valid_df = split_train_valid(train_df)
#     test_df = get_dataframe(x_test, y_test, "test", sample_size=sample_size // 10)

#     # Data augmentation for training set
#     train_datagen, valid_datagen, test_datagen = get_img_generators()
#     train_generator = to_generator(train_datagen, train_df, dl.train_dir)
#     valid_generator = to_generator(valid_datagen, valid_df, dl.train_dir)
#     test_generator = to_generator(test_datagen, test_df, dl.test_dir)
    
#     train_len = len(x_train)
#     valid_len = len(valid_df)
    test_len = len(x_test)
    
    if is_final_model:
        # If it's a finalized model, train with a larger epochs
        trained_model = train(x_train, y_train)

        evaluation = trained_model.evaluate(
            x=x_test, y=y_train)
        y_hat = trained_model.predict(
            x=x_test)
        
        print(evaluation)
        compare_results(y_test, y_hat)

        return evaluation, y_hat, y_test
    
#     else:
#         train_many(train_generator, valid_generator, train_len, valid_len)

In [ ]:
res = main()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 2048)]            0         
_________________________________________________________________
d1 (Dense)                   (None, 512)               1049088   
_________________________________________________________________
bn1 (BatchNormalization)     (None, 512)               2048      
_________________________________________________________________
d2 (Dense)                   (None, 256)               131328    
_________________________________________________________________
bn2 (BatchNormalization)     (None, 256)               1024      
_________________________________________________________________
d3 (Dense)                   (None, 55)                14135     
Total params: 1,197,623
Trainable params: 1,196,087
Non-trainable params: 1,536
_____________________________________________